In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains


import pandas as pd
import re
import getpass
from bs4 import BeautifulSoup
import json
import os
import time

import pyautogui

In [2]:
DAY = '08'
JUPYTERNB = f'https://github.com/dsmdavid/AdventCode2020/blob/main/Code/Python/Day_{DAY}/Day_{DAY}.ipynb'



In [3]:
with open('../../ignore/convo.json','r') as f:
    keys = json.loads(f.read())

In [4]:
curr_dir = os.getcwd()
chromedriver = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'Resources/chromedriver')

In [5]:
import platform
# Win or mac?
curr_sys = platform.system()
if curr_sys != 'Darwin':
    chromedriver = chromedriver + '.exe'

In [6]:
# chromedriver = r'C:\Users\dsmda\Downloads\chromedriver_win32\chromedriver.exe'

In [7]:

class WebScraping(object):
   
    """
    selectors are defined to here to for use in raising an error downstream
    """
    selectors = ['ID'
            ,'NAME'
            ,'XPATH'
            ,'LINK_TEXT'
            ,'PARTIAL_LINK_TEXT'
            ,'TAG_NAME','CLASS_NAME'
            ,'CSS_SELECTOR']
   
    def __init__(self):
        pass

    @staticmethod
    def driv(dloaddirectory="C:\\Users\\" + str(getpass.getuser()) + "\\Downloads"):
        """
        Recommended use is to store the result of this function as a variable
        e.g. driver = WebScraping.driv() be mindful that if you import the
        module with an alias e.g. inport WebScraping as WS then you would
        use the alias preceding .driv() in this example WS.driv()
       
        Initialises an instance of a driver, by default the download directory
        is set to the downloads folder on the C: drive but to alter this simply
        pass the optional variable dloaddirectory with the corresponding path
        """
        ## define chrome webdriver settings
        capabilities = { 'chromeOptions':
                        { 'useAutomationExtension': False}
                       }
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")
        #options.add_argument("--window-size=1920,1080")
        prefs = { 'download.default_directory': dloaddirectory }
        options.add_experimental_option('prefs', prefs)
        options.add_experimental_option("useAutomationExtension", False);
        return webdriver.Chrome(executable_path = chromedriver, options=options)    
   
   
    @classmethod
    def selection(cls, el_type, target, driver):
        """
        el_type is the element type you are looking to capture e.g. XPATH
        Target is the path to that element
        driver is the instance that you have declared when calling the driv() function
       
        This method is used to return the value
        of the taret selected, by compounding with .click()
        to you can interact with the target on_click event
       
        Raises:
        Errors if a element is passed that is not within the following list
        ID, NAME, XPATH, LINK_TEXT, PARTIAL_LINK_TEXT , TAG_NAME, CLASS_NAME, CSS_SELECTOR
       
        """
        #Throw error if element selected is outside of the accepted elements
        if el_type not in cls.selectors:
            raise Exception('{} is not an accepted element, only the following are accepted elements - ID, NAME, XPATH, LINK_TEXT, PARTIAL_LINK_TEXT , TAG_NAME, CLASS_NAME, CSS_SELECTOR'.format(el_type))
        # Standard elements defined in the Selenium locating elements documentation
        if el_type.upper() == 'ID':
            return driver.find_element_by_id(target)
        if el_type.upper() == 'LINK_TEXT':
            return driver.find_element_by_link_text(target)
        if el_type.upper() == 'NAME':
            return driver.find_element_by_name(target)
        if el_type.upper() == 'XPATH':
            return driver.find_element_by_xpath(target)
        if el_type.upper() == 'LINK_TEXT':
            return driver.find_element_by_link_text(target)
        if el_type.upper() == 'PARTIAL_LINK_TEXT':
            return driver.find_element_by_partial_link_text(target)
        if el_type.upper() == 'TAG_NAME':
            return driver.find_element_by_tag_name(target)
        if el_type.upper() == 'CLASS_NAME':
            return driver.find_element_by_class_name(target)
        if el_type.upper() == 'CSS_SELECTOR':
            return driver.find_element_by_class_css_selector(target)

    @staticmethod
    def getTables(driver):
        """
        Returns all tables on the displayed page as a list
       
        Can be iterated thorugh if all tables are required
        or sliced if you are looking for a certain table
        within a page
        """
        return BeautifulSoup(driver.page_source).find_all('table')
   
    @staticmethod
    def getLinks(html):
        """
        Returns all of the hrefs (Links) from a HTML string
       
        Recommended use is in conjuction with getTables to extract
        all links from a html string
       
        """
        return html.find_all('a')

## Screenshot Jupyter notebook in github

In [8]:
def get_jupyter_screenshot(DAY):
    
    JUPYTERNB = f'https://github.com/dsmdavid/AdventCode2020/blob/main/Code/Python/Day_{DAY}/Day_{DAY}.ipynb'
    art_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'Art')

    driver = WebScraping.driv(art_path)

    driver.get(JUPYTERNB)
    element_present = EC.presence_of_element_located((By.XPATH, '/html/body/div/div[2]'))
    WebDriverWait(driver, 10).until(element_present)
    time.sleep(1)

    ele = driver.find_element_by_xpath('/html/body/div/div[2]') 
    ele2 = driver.find_element_by_xpath('/html/body/div[5]/div[1]/ul')

    actions = ActionChains(driver)
    actions.move_to_element(ele).perform()

    height = ele2.location['y'] -  ele.location_once_scrolled_into_view['y']
    time.sleep(0.3)

    driver.set_window_size(ele.size['width'], height)
    driver.save_screenshot(f'../../Art/Day_{DAY}.png')


    driver.close()



In [9]:
get_jupyter_screenshot(DAY)
time.sleep(1)
get_jupyter_screenshot(DAY)

## Post to Convo:

In [10]:
def valid_name(filename):
    return  filename.startswith(f'day_{DAY}') & ((filename.endswith('yxmd') or filename.endswith('yxzp')))

def get_workflow():
    curr_dir = os.getcwd()
    alteryx_dir = os.path.join(os.path.dirname(os.getcwd()), 'Alteryx')
    os.chdir(os.path.join(alteryx_dir, f'Day_{DAY}'))
    files = os.listdir()
    workflows = list(filter(valid_name, files))
    workflows = {w: os.path.getmtime(w) for w in workflows}
    t = 0
    file = ''
    for k,v in workflows.items():
           if v > t:
                t, file = v, k
    k = os.path.join(os.getcwd(),k)
    os.chdir(curr_dir)
    return k

In [11]:
options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
#options.add_argument("--headless")

driver = webdriver.Chrome(executable_path =chromedriver,  options=options)

driver.get(f'https://app.convo.com/app/login/')

email = driver.find_element(By.ID,'email')
email.send_keys(keys['username'])

password = driver.find_element(By.ID, 'password')
password.send_keys(keys['password'])

btn = driver.find_element(By.ID, 'btnSignIn')
btn.click()

mypost = "https://app.convo.com/#/v1/acc-26f67bc7-4b15-4825-e053-760e880a4be5/apps/6/messages/0WwMN71JYuz8dQhnfSQ3V9QpIWUFxN#title=AoC2020%252520BaseA%252520(and%252520Python%252520as%252520%252520long%252520as%252520I'm%252520able%252520to)"
driver.get(mypost)

element_present = EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/cnv-app-cont/cnv-notes-app/div/ng-include/section/div[2]/ng-include/ul/li[2]'))
WebDriverWait(driver, 10).until(element_present)
comment = driver.find_element_by_xpath('/html/body/div[1]/cnv-app-cont/cnv-notes-app/div/ng-include/section/div[2]/ng-include/ul/li[2]')

comment.click()

actions = ActionChains(driver)
actions.move_to_element(comment).perform()

element = driver.switch_to.active_element

art_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'Art')

jupyter_screenshot = os.path.join(art_path, f'Day_{DAY}.png')
alteryx_screenshot = os.path.join(art_path, f'Alteryx_Day_{DAY}.png')
alteryx_workflow = get_workflow()

attachments = [jupyter_screenshot, alteryx_screenshot, alteryx_workflow]

time.sleep(1)

In [13]:
pyautogui.getAllTitles()

['',
 'take_screenshot - Jupyter Notebook - Google Chrome',
 "AoC2020 BaseA (and Python as long as I'm able to) - Convo - Google Chrome",
 'Jupyter Notebook (Tests)',
 'Command Prompt',
 'Malwarebytes Tray Application',
 'Art',
 'Slack | bi_chapter | Infinite Lambda | 1 new item',
 'Advent of Code 2020 - BaseA Style (Day 8) - Alteryx Community - Google Chrome',
 'Snip & Sketch',
 'Snip & Sketch',
 'Alteryx_Day_08 - Paint',
 '',
 'Calculator',
 'Calculator',
 '',
 'Microsoft Text Input Application',
 '',
 '',
 '',
 '',
 '',
 'Microsoft Store',
 'Microsoft Store',
 'Snip & Sketch',
 'Snip & Sketch',
 'GitHub Desktop',
 'Program Manager']

In [15]:
pyautogui.getWindowsWithTitle(title)

[Win32Window(hWnd=2296266)]

In [16]:

title = [title for title in pyautogui.getAllTitles() if 'AoC2020' in title][0]
pyautogui.getWindowsWithTitle(title)

for attachment in attachments:
    if attachment:
        attach = driver.find_element_by_class_name('file-chooser')
        time.sleep(5)
        actions = ActionChains(driver)
        actions.move_to_element(attach).perform()

        attach.click()
        time.sleep(2)

        pyautogui.write(attachment) 
        pyautogui.press('enter')

comment.click()

actions = ActionChains(driver)
actions.move_to_element(comment).perform()

element = driver.switch_to.active_element

template = f'''Day_{DAY}:\t\tJupyter Notebook ({JUPYTERNB})'''

element.send_keys(template)
time.sleep(1)
actions = ActionChains(driver)
actions.move_to_element(comment).perform()
element.send_keys(' ')

time.sleep(1)
element.send_keys('\n')

In [ ]:
pyautogui.getAllTitles()

In [ ]:
pyautogui.getWindowsWithTitle(pyautogui.getAllTitles()[2])

In [ ]:
len(pyautogui.getAllTitles()), len(pyautogui.getAllWindows())

In [ ]:
driver.close()

In [ ]:
driver.close()

output = os.popen('wmic process get description, processid').read()
output